In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import functools as ft
import numpy as np
import pandas as pd
from time import sleep
import re
from pprint import pprint
import time
import requests
from bs4 import BeautifulSoup


In [2]:
today_mm, today_dd = list(map(int,time.strftime('%m.%d').split('.')))
today_mm, today_dd

(4, 3)

In [3]:
year_max, year_min = 2022,2000
year_pages = []
print('Range : %d ~ %d'%(year_min,year_max))
for info in BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie_open.nhn'),'html.parser').find('table').select('td'):
    if year_min <= int(info.text[:4]) <= year_max:
        year, pages = info.text[:4], str(np.ceil(float(info.text[7:-1])/20).astype(int))
        print('Year '+year+' - '+pages+' pages')
        year_pages.append((year,pages))
year_pages

Range : 2000 ~ 2022
Year 2022 - 11 pages
Year 2021 - 38 pages
Year 2020 - 37 pages
Year 2019 - 43 pages
Year 2018 - 50 pages
Year 2017 - 53 pages
Year 2016 - 59 pages
Year 2015 - 71 pages
Year 2014 - 76 pages
Year 2013 - 61 pages
Year 2012 - 44 pages
Year 2011 - 41 pages
Year 2010 - 29 pages
Year 2009 - 24 pages
Year 2008 - 28 pages
Year 2007 - 26 pages
Year 2006 - 24 pages
Year 2005 - 21 pages
Year 2004 - 38 pages
Year 2003 - 20 pages
Year 2002 - 18 pages
Year 2001 - 17 pages
Year 2000 - 19 pages


[('2022', '11'),
 ('2021', '38'),
 ('2020', '37'),
 ('2019', '43'),
 ('2018', '50'),
 ('2017', '53'),
 ('2016', '59'),
 ('2015', '71'),
 ('2014', '76'),
 ('2013', '61'),
 ('2012', '44'),
 ('2011', '41'),
 ('2010', '29'),
 ('2009', '24'),
 ('2008', '28'),
 ('2007', '26'),
 ('2006', '24'),
 ('2005', '21'),
 ('2004', '38'),
 ('2003', '20'),
 ('2002', '18'),
 ('2001', '17'),
 ('2000', '19')]

In [5]:
year_max, year_min = 2022,2000
year_pages = []
print('Range : %d ~ %d'%(year_min,year_max))
for info in BeautifulSoup(urlopen('https://book.naver.com/bestsell/bestseller_list.nhn'),'html.parser').find('table').select('td'):
    if year_min <= int(info.text[:4]) <= year_max:
        year, pages = info.text[:4], str(np.ceil(float(info.text[7:-1])/20).astype(int))
        print('Year '+year+' - '+pages+' pages')
        year_pages.append((year,pages))
year_pages


Range : 2000 ~ 2022


HTTPError: HTTP Error 500: Internal Server Error

In [ ]:
csvdata,csvtotal = [], 0
for year, pages in year_pages:
    for page in range(1,int(pages)+1):
        try:
            soup = BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?open='+year+'&page='+str(page)),'html.parser')
            collected, total = 0, 0
            for link in soup.find('ul','directory_list').select('li > a'):
                err_msg = ''
                try:
                    date = re.findall('\d{2}.\d{2}',link.text)

                    if 'href' in link.attrs and 'class' not in link.attrs:
                        basic_page = 'https://movie.naver.com'+link.attrs['href']
                        data = BeautifulSoup(urlopen(basic_page),'html.parser')
                        total += 1

                        # title
                        title = data.find('h3','h_movie').select('a')[0].text

                        msg = 'error at date'
                        # date
                        date = data.find('dl','info_spec').find('dd').find_all('span')[-1].find_all('a')[-1].text[1:]
                        
                        msg = 'error at rating & voting'
                        # rating, vote_count
                        rating = str(ft.reduce(lambda x,y:x+y,[val.text for val in data.find('div','score score_left').select('div > a > em')[:4]]))
                        vote_count = data.find('div','score score_left').find('div','ly_count').em.text
                            
                        msg = 'error at genre'
                        # genre
                        genre = '|'.join(list(map(lambda x: x.text, filter(lambda x: 'genre' in x.attrs['href'], data.find('dl','info_spec').select('dd > p > span > a')))))
                        #genre = genre.replace('멜로/로맨스','로맨스')
                        
                        msg = 'error at plot'
                        #plot
                        plot = data.find_all('p','con_tx')[0].text
                        
                        msg = 'error at actors'
                        # actors
                        detail_page = basic_page.replace('basic','detail')
                        detail_soup = BeautifulSoup(urlopen(detail_page))

                        main_act,supp_act = '',''
                        main_num = sum([1 if line.text == '주연' else 0 for line in detail_soup.find_all('em','p_part')])
                        if main_num > 0:
                            for i,actor in enumerate(detail_soup.find('ul','lst_people').find_all('a','k_name')):
                                if i<main_num:
                                    main_act += actor.text+'|'
                                else:
                                    supp_act += actor.text+'|'
                            main_act,supp_act = main_act[:-1],supp_act[:-1]

                        msg = 'error at img'
                        #img_url
                        img_url = data.find('div','poster').find('img').attrs['src']

                        # data check => save
                        if rating != '0.00' and genre != '' and len(date) > 0:
                            '''
                            영화제목, 장르, 연도, 날짜, 평점, 투표수, 주연, 조연, url, 이미지링크
                            '''
                            csvdata.append([title, genre, year, date, rating, vote_count,
                                           plot, main_act, supp_act, basic_page,img_url])
                            collected += 1
                            csvtotal += 1
                        
                except Exception as e:
                    print('Error at title({}) & {}'.format(title,msg))
            print('year {0} page {1}/{2} - crawling {3}/{4} completed - {5}'.format(year,page,pages,collected,total,csvtotal))
            #sleep(3)
        except Exception as e:
            print('year {0} page {1}/{2} - crawling {3}/{4} failed - {5}'.format(year,page,pages,collected,total,csvtotal))
        if page % 10 == 0:
            print('Check per 10 pages - Recently appended 10 data')
            pprint(csvdata[-5:])

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


# 네이버의 베스트셀러 웹페이지를 가져옵니다.
html = urlopen('https://book.naver.com/bestsell/bestseller_list.nhn')
bsObject = BeautifulSoup(html, "html.parser")


# 책의 상세 웹페이지 주소를 추출하여 리스트에 저장합니다.
book_page_urls = []
for index in range(0, 25):
    dl_data = bsObject.find('dt', {'id':"book_title_"+str(index)})
    link = dl_data.select('a')[0].get('href')
    book_page_urls.append(link)



# 메타 정보와 본문에서 필요한 정보를 추출합니다.  
for index, book_page_url in enumerate(book_page_urls):
    html = urlopen(book_page_url)
    bsObject = BeautifulSoup(html, "html.parser")


    title = bsObject.find('meta', {'property':'og:title'}).get('content')
    author = bsObject.find('dt', text='저자').find_next_siblings('dd')[0].text.strip()
    image = bsObject.find('meta', {'property':'og:image'}).get('content')
    url = bsObject.find('meta', {'property':'og:url'}).get('content')

    dd = bsObject.find('dt', text='가격').find_next_siblings('dd')[0]
    salePrice = dd.select('div.lowest strong')[0].text
    originalPrice = dd.select('div.lowest span.price')[0].text

    print(index+1, title, author, image, url, originalPrice, salePrice)


HTTPError: HTTP Error 500: Internal Server Error

In [9]:
url = 'https://book.naver.com/search/search.nhn'
params = {'sm':'sta_hty.book', 'sug':'', 'where':'nexearch', 'query':'bigdata'}

In [12]:
# GET 요청
response = requests.get(url, params=params)
status_code = response.status_code
print(status_code)
if status_code == 200:
    text = response.text

200


In [13]:
soup = BeautifulSoup(text)

In [14]:
book_all = soup.find(id='searchBiblioList')

In [15]:
# 책 한 권 정보
find_one = soup.find(class_='thumb_type thumb_type2')
#find_one = soup.find(attrs={'class':'thumb_type thumb_type2'})
############아니니닫find_one = soup.select_one('/thumb_type thumb_type2')

print(find_one)

<div class="thumb_type thumb_type2">
				<a class="N=a:bls.thumb,r:1,i:98000001_00000000000000000150246D" href="http://book.naver.com/bookdb/book_detail.naver?bid=22029421" target="_blank">
						<img alt="Big Data and Blockchain for Service Operations Management" onerror="this.src='https://ssl.pstatic.net/static/book/image/noimg3.gif';" src="https://bookthumb-phinf.pstatic.net/cover/220/294/22029421.jpg?type=m1&amp;udate=Tue Mar 15 11:45:49 KST 2022"/><span class="mask"><span class="bg1"></span><span class="bg2"></span></span>
				</a>
        	</div>


In [16]:
# 책 각각, 전체
book_all_li_all = book_all.select('li')
# print(book_all_li_all)

# 책 한 권
book_all_li_one = book_all.select_one('li')
print(book_all_li_one)

<li style="position:relative;">
	<div class="thumb type_search">
        	<div class="thumb_type thumb_type2">
				<a class="N=a:bls.thumb,r:1,i:98000001_00000000000000000150246D" href="http://book.naver.com/bookdb/book_detail.naver?bid=22029421" target="_blank">
						<img alt="Big Data and Blockchain for Service Operations Management" onerror="this.src='https://ssl.pstatic.net/static/book/image/noimg3.gif';" src="https://bookthumb-phinf.pstatic.net/cover/220/294/22029421.jpg?type=m1&amp;udate=Tue Mar 15 11:45:49 KST 2022"/><span class="mask"><span class="bg1"></span><span class="bg2"></span></span>
				</a>
        	</div>
        </div>
	<dl style="width:654px">
	<dt>
		<a class="N=a:bls.title,r:1,i:98000001_00000000000000000150246D" href="http://book.naver.com/bookdb/book_detail.naver?bid=22029421" target="_blank"><strong>Big</strong> <strong>Data</strong> and Blockchain for Service Operations Management</a> </dt>
	
	<dd class="txt_block">
		<a class="N=a:bls.author,r:1,i:87680" hr

In [41]:
book_all_li_one.span['txt_gl']

KeyError: 'txt_gl'

In [17]:
# 책 bid 가져오기
bid_one = book_all_li_one.a['href'].split('=')[1]
# print(bid_one)

# 책 bid 여러개 가져오기
# for book_all_li_one in book_all_li_all:
#     print(book_all_li_one.a['href'].split('=')[1])
    
# 책 bid 여러개 가져와서 list에 담기
bid_list = []
for book_all_li_one in book_all_li_all:
    b_id = book_all_li_one.a['href'].split('=')[1]
    bid_list.append(b_id)
print(bid_list)

['22029421', '21975341', '16338249', '22029451', '21468306', '22008668', '22146947', '13587569', '21507679', '21392892', '22080780', '15516543', '21360835', '21152310', '21400248', '20595866', '21365558', '18063252', '18001381', '21975874']


In [23]:
book_all_li_one.img['alt']

'Body Area Networks. Smart IoT and Big Data for Intelligent Health Management'

In [18]:
# 책 제목 하나 가져오기
title_one = book_all_li_one.img['alt']
#print(title_one)

# 책 전체 가져와서 list에 담기
title_list = []
for book_all_li_one in book_all_li_all:
    b_title = book_all_li_one.img['alt']
    title_list.append(b_title)
print(title_list)

['Big Data and Blockchain for Service Operations Management', 'Big Data in Energy Economics', 'BIG DATA를 활용한 K-뷰티경영학', 'Big Data - BigData 2021', 'Big Data, IoT, and Machine Learning: Tools and Applications', "Decoding Talent: How AI and Big Data Can Solve Your Company's People Puzzle", 'Adaptive Big Data Model for Sentiment Analysis of Social Media Applications', '빅데이터', 'Logic-Driven Traffic Big Data Analytics', 'Big Data Analytics', 'Cultural Heritage Storytelling, Engagement and Management in the Era of Big Data and the Semantic Web', 'KNIME을 활용한 Big Data분석', 'The Data Revolution: A Critical Analysis of Big Data, Open Data and Data Infrastructures', 'Big data, ascendance controversee du marketing hyperpersonnalise', 'Software Engineering in IoT, Big Data, Cloud and Mobile Computing', 'Radiomics and Its Clinical Application: Artificial Intelligence and Medical Big Data', 'Proceedings of the International Conference on Big Data, IoT, and Machine Learning', 'Big Data', 'Big Data', 'Bo

In [37]:
a=book_all_li_one.select_one('dd.searchDescrition_22029421').text

AttributeError: 'NoneType' object has no attribute 'text'

In [24]:
book_text = book_all_li_one.select_one('dd.txt_block').text
print(book_text)
book_text = book_text.replace('\n','').replace('\r','')
book_text = book_text.replace('\t','').replace('\xa0','')
book_text_list = book_text.split('|')
book_author = book_text_list[0]
book_publisher = book_text_list[1]
book_pubdate = book_text_list[2]
print(book_author,book_publisher,book_pubdate)




		Zoha, Ahmed(EDT) 저 | SpringerNatureSwitzerlandAG | 2022.03.14
Zoha, Ahmed(EDT) 저  SpringerNatureSwitzerlandAG 2022.03.14


In [25]:
book_text

'Zoha, Ahmed(EDT) 저 |SpringerNatureSwitzerlandAG|2022.03.14'

In [28]:
# 책 한 권 가격 정보 가져오기
book_txt_desc = book_all_li_all[1].select_one('dd.txt_desc') # 전체 정보 중에 1index 값 정보 가져오기
#print(book_txt_desc)

price_org = book_txt_desc.select_one('strike').text
#print(price_org)

price_org = price_org.split('원')[0]
print(price_org)

price_disc = book_txt_desc.select_one('em.price').text
#print(price_disc)

price_disc = price_disc.split('원')[0]
print(price_disc)

102470
91660


In [29]:
book_txt_desc

<dd class="txt_desc">
		<div class="review_point">
					<span style="width:0.0%;">별점</span>
				</div>
				0.0<span class="blind">점</span>
				<span class="bar"> | </span>
				<a class="N=a:bls.review,r:2,i:" href="/bookdb/review.naver?bid=21975341">네티즌리뷰</a>
					0건
				<span class="bar">|</span>
				<a class="N=a:bls.bookbuy,r:2,i:98000001_0000000000000000014F512D" href="javascript:showBuyLayerByBid('21975341')" id="buy_btn_21975341" onclick="return showAdultLayer('21975341', 'false', 'false', 'false');"><img alt="도서구매" class="btn v2" height="20" id="btn_buy_img_21975341" src="https://ssl.pstatic.net/static/book/image/btn_book_buy.gif" title="구매 가능한 도서입니다." width="48"/></a>
					 <strike>102470원</strike> → <em class="price">91660원(-11%)</em>
				</dd>

In [27]:
price_list = list()

for book_all_li_one in book_all_li_all:
    book_txt_desc = book_all_li_one.select_one('dd.txt_desc')
    price_org = book_txt_desc.select_one('strike')
    price_disc = book_txt_desc.select_one('em.price')
    # pirce가 None인 경우 해결하기
    if price_org == None:
        price_org = 0
    else:
        price_org = price_org.text.split('원')[0] # text만 뽑아서 작업
        
    if price_disc == None:
        price_disc = 0
    else:
        price_disc = price_disc.text.split('원')[0]
        
    price_list.append((price_org, price_disc))
    
print(price_list)

[(0, 0), ('102470', '91660'), ('21000', '20370'), (0, 0), ('102000', '95620'), (0, 0), (0, 0), ('13000', '11700'), ('225470', '201690'), ('120720', '109990'), (0, 0), ('25000', '22500'), ('174030', '164520'), ('68420', '64680'), ('182220', '165010'), (0, 0), (0, 0), (0, 0), ('28250', '28120'), ('133220', '119160')]


In [45]:
book_all_li_one.select_one('dd.searchDescrition')

In [50]:
!pip install webdriver_manager

In [72]:
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

import time

import csv

url = "https://book.naver.com/category/index.nhn?cate_code=100010&tab=top100&list_type=list&sort_type=publishday&page=2"




====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/camilleyoon/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [81]:
book_page_urls

['http://book.naver.com/bookdb/book_detail.naver?bid=18990871',
 'http://book.naver.com/bookdb/book_detail.naver?bid=18926010',
 'http://book.naver.com/bookdb/book_detail.naver?bid=18903895',
 'http://book.naver.com/bookdb/book_detail.naver?bid=18870190',
 'http://book.naver.com/bookdb/book_detail.naver?bid=18262558',
 'http://book.naver.com/bookdb/book_detail.naver?bid=18123571',
 'http://book.naver.com/bookdb/book_detail.naver?bid=17932329',
 'http://book.naver.com/bookdb/book_detail.naver?bid=17932356',
 'http://book.naver.com/bookdb/book_detail.naver?bid=17470676',
 'http://book.naver.com/bookdb/book_detail.naver?bid=17467599',
 'http://book.naver.com/bookdb/book_detail.naver?bid=17357620',
 'http://book.naver.com/bookdb/book_detail.naver?bid=17603841',
 'http://book.naver.com/bookdb/book_detail.naver?bid=16851325',
 'http://book.naver.com/bookdb/book_detail.naver?bid=16687560',
 'http://book.naver.com/bookdb/book_detail.naver?bid=16565162',
 'http://book.naver.com/bookdb/book_deta

In [82]:
file = open('bookdb_소설_나라별.csv','w',encoding='utf-8', newline='')

csvfile = csv.writer(file)

csvfile.writerow(["순위", "책이름", "저자", "줄거리","이미지"])



for index, book_page_url in enumerate(book_page_urls):

    

    driver.get(book_page_url)

    bsObject = BeautifulSoup(driver.page_source, 'html.parser')


    title = bsObject.find('meta', {'property':'og:title'}).get('content')

    author = bsObject.find('dt', text='저자').find_next_siblings('dd')[0].text.strip()

    image = bsObject.find('meta', {'property':'og:image'}).get('content')

    description = bsObject.find('meta', {'property' : 'og:description'}).get('content')

    url = bsObject.find('meta', {'property':'og:url'}).get('content')

    dd = bsObject.find('dt', text='가격').find_next_siblings('dd')[0]


    print(index+1, title, author, image)

    

    csvfile.writerow([index+1, title, author, description, image])


file.close()

1 호랑이를 덫에 가두면 태 켈러 https://bookthumb-phinf.pstatic.net/cover/189/908/18990871.jpg?type=m5
2 불편한 편의점 김호연 https://bookthumb-phinf.pstatic.net/cover/189/260/18926010.jpg?type=m5
3 달까지 가자 장류진 https://bookthumb-phinf.pstatic.net/cover/189/038/18903895.jpg?type=m5
4 제12회 젊은작가상 수상작품집 전하영, 김멜라, 김지연, 김혜진, 박서련 https://bookthumb-phinf.pstatic.net/cover/188/701/18870190.jpg?type=m5
5 클라라와 태양 가즈오 이시구로 https://bookthumb-phinf.pstatic.net/cover/182/625/18262558.jpg?type=m5
6 순례 주택 유은실 https://bookthumb-phinf.pstatic.net/cover/181/235/18123571.jpg?type=m5
7 듄 1 프랭크 허버트 https://bookthumb-phinf.pstatic.net/cover/179/323/17932329.jpg?type=m5
8 듄 1-6 세트 프랭크 허버트 https://bookthumb-phinf.pstatic.net/cover/179/323/17932356.jpg?type=m5
9 블랙 쇼맨과  이름 없는 마을의 살인 히가시노 게이고 https://bookthumb-phinf.pstatic.net/cover/174/706/17470676.jpg?type=m5
10 일인칭 단수 무라카미 하루키 https://bookthumb-phinf.pstatic.net/cover/174/675/17467599.jpg?type=m5
11 인생은 소설이다 기욤 뮈소 https://bookthumb-phinf.pstatic.net/cover/173/576/17357620.jpg?type=

In [87]:
import pandas as pd

In [115]:
df=pd.DataFrame(columns=["순위", "책이름", "저자", "줄거리"])

In [101]:
df

,순위,책이름,저자,줄거리,이미지


In [144]:
li=[1,2,3,4]

In [127]:
df.append(li)

/var/folders/pm/x15q7lz11ks31sdtpqsy5z1w0000gn/T/ipykernel_3015/1453041757.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(li)


,순위,책이름,저자,줄거리,0
0,NaN,NaN,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,2.0
2,NaN,NaN,NaN,NaN,3.0
3,NaN,NaN,NaN,NaN,4.0


In [129]:
df

,순위,책이름,저자,줄거리


In [130]:
df.append(1)

/var/folders/pm/x15q7lz11ks31sdtpqsy5z1w0000gn/T/ipykernel_3015/294144530.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(1)


TypeError: cannot concatenate object of type '<class 'int'>'; only Series and DataFrame objs are valid

In [202]:
df[200:220]

[[201,
  '감시와 처벌',
  '미셀 푸코',
  '처벌의 종류와 감시방법, 감옥의 탄생과정을 심층적으로 고찰한 책. 감옥과 처벌의 내면적, 외형적 변화를 통해 근대 이후의 행형사법제도와 권력의 관계를 규명하고 있다．감옥을 정점으로 하는 감시 처벌의 기구인 가정, 학교, 군대, 병원, 공장 등을 분석하고 사실상 근대사회를 감금사회, 관리사회, 처벌사회, 감시사회로 이해하였다. 이 책은 외형적인 형벌의 변화도 결국 권력의 자기보호책이었음을 다양한 자료를 통해 보여준다．또한 보다 정교해진 행형기술이 사회전체를 통제하고 조종하는 국가관리술로 발전했음에 주목하고 있다. 감옥, 소년원 등에서 주로 활용됐던 복종, 시간표에 의한 인력관리, 규율에 대한 강조가 군대, 학교, 병원, 공장 등 사회전체에 적용되었다고 말한다.'],
 [202,
  '18세기 조선 지식인의 발견',
  '정민',
  '한국 문화사에서 가장 매력적이고 생동감 넘치는 18세기!이 시기에 도대체 무슨 일이 일어났던 것일까?〈18세기 조선 지식인의 발견〉은 조선 후기 지식 패러다임의 변화와 문화 변동을 살펴보는 책이다. 문학을 넘어 문화사 전반으로 글쓰기와 사유의 폭을 넓혀가고 있는 저자가 지난 2001년부터 최근까지 7년간 18세기에 대해 탐구한 결과를 담았다. 18세기 문화 변동기의 문화 현상을 다룬 13편의 논문으로 구성되어 있다. 저자는 18세기가 우리 문화사에서 아주 매력적이고 생동감 넘치는 시기라고 이야기한다. 이 책은 지식, 자의식, 글쓰기, 감각, 취미 등이 새롭게 구성된 18세기를 문화사적으로 정리하였다. 이 시기 조선 지식인의 지적 경향과 문화 변동을 아우른 문화사로 18세기의 특징적 문화 현상, 18세기 조선..'],
 [203,
  '인문으로 읽는 주역',
  '신원봉',
  '고금의 뛰어난 해석을 모아 주역의 본뜻을 구한다!『인문으로 읽는 주역』. 주역은 동양 고전 중에서도 가장 읽기 어렵고 추상적이다. 당대의 뛰어난 학자라면 주석서 한 권을 냈을 정도로 그 영향력이 대단

In [151]:

ur = "https://book.naver.com/category/index.nhn?cate_code=100010&tab=top100&list_type=list&sort_type=publishday&page="
for i in range(3,4):
    url = ur+str(i+1)
    print(url)
    driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.implicitly_wait(10)
    driver.get(url)

    bsObject = BeautifulSoup(driver.page_source, 'html.parser')
    book_page_urls = []
    teni=int(i)*20
    for index in range(0, 20):
    
        dl_data = bsObject.find('dt', {'id':"book_title_"+str(index)})

        link = dl_data.select('a')[0].get('href')

        book_page_urls.append(link)
        file = open('소설정보3.csv','w',encoding='utf-8', newline='')

    csvfile = csv.writer(file)

    csvfile.writerow(["순위", "책이름", "저자", "줄거리","이미지"])


    try:
        for index, book_page_url in enumerate(book_page_urls):



            driver.get(book_page_url)

            bsObject = BeautifulSoup(driver.page_source, 'html.parser')


            title = bsObject.find('meta', {'property':'og:title'}).get('content')

            author = bsObject.find('dt', text='저자').find_next_siblings('dd')[0].text.strip()

    #         image = bsObject.find('meta', {'property':'og:image'}).get('content')

            description = bsObject.find('meta', {'property' : 'og:description'}).get('content')

            url = bsObject.find('meta', {'property':'og:url'}).get('content')

            dd = bsObject.find('dt', text='가격').find_next_siblings('dd')[0]
            toin=teni+index

            print(toin+1, title, author)

            csvfile.writerow([toin+1, title, author, description, image])


        file.close()
    except:
        pass
        



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome


https://book.naver.com/category/index.nhn?cate_code=100010&tab=top100&list_type=list&sort_type=publishday&page=4


Driver [/Users/camilleyoon/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


61 봉제인형 살인사건 다니엘 콜
62 용의자 X의 헌신 히가시노 게이고
63 노르웨이의 숲 무라카미 하루키
64 바깥은 여름 김애란
65 아몬드 손원평
66 아몬드 손원평
67 돌이킬 수 없는 약속 야쿠마루 가쿠
68 여름은 오래 그곳에 남아 마쓰이에 마사시
69 종의 기원 정유정
70 사서함 110호의 우편물 이도우
71 어린 왕자 생텍쥐페리
72 앵무새 죽이기 하퍼 리
73 멋진 신세계 올더스 헉슬리
74 구의 증명 최진영
75 소년이 온다 한강
76 외계인 인터뷰 로렌스 R. 스펜서
77 모순 양귀자
78 나미야 잡화점의 기적 히가시노 게이고
79 노인과 바다 어니스트 헤밍웨이
80 페스트 알베르 카뮈


In [213]:
len(df)

100

In [214]:
df2=pd.DataFrame(data=df,columns=["순위", "책이름", "저자", "줄거리"])

In [215]:
df2.to_csv('./data/고전.csv')

In [210]:
ur='https://book.naver.com/category/index.naver?cate_code=100020&tab=top100&list_type=list&sort_type=publishday&page='

In [211]:
df=[]

In [212]:

for i in range(0,5):
    url = ur+str(i+1)
    print(url)
    driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.implicitly_wait(10)
    driver.get(url)

    bsObject = BeautifulSoup(driver.page_source, 'html.parser')
    book_page_urls = []
    teni=int(i)*20
    for index in range(0, 20):
    
        dl_data = bsObject.find('dt', {'id':"book_title_"+str(index)})

        link = dl_data.select('a')[0].get('href')

        book_page_urls.append(link)
        file = open('소설정보3.csv','w',encoding='utf-8', newline='')

    csvfile = csv.writer(file)

    csvfile.writerow(["순위", "책이름", "저자", "줄거리","이미지"])


    try:
        for index, book_page_url in enumerate(book_page_urls):



            driver.get(book_page_url)

            bsObject = BeautifulSoup(driver.page_source, 'html.parser')


            title = bsObject.find('meta', {'property':'og:title'}).get('content')

            author = bsObject.find('dt', text='저자').find_next_siblings('dd')[0].text.strip()

    #         image = bsObject.find('meta', {'property':'og:image'}).get('content')

            description = bsObject.find('meta', {'property' : 'og:description'}).get('content')

            url = bsObject.find('meta', {'property':'og:url'}).get('content')

            dd = bsObject.find('dt', text='가격').find_next_siblings('dd')[0]
            toin=teni+index

#             print(toin+1, title, author)
            li=[toin+1, title, author, description]
            df.append(li)
#             print(df)
#             csvfile.writerow([toin+1, title, author, description, image])


        file.close()
    except:
        pass
        



====== WebDriver manager ======


https://book.naver.com/category/index.naver?cate_code=100020&tab=top100&list_type=list&sort_type=publishday&page=1


Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/camilleyoon/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


====== WebDriver manager ======


https://book.naver.com/category/index.naver?cate_code=100020&tab=top100&list_type=list&sort_type=publishday&page=2


Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/camilleyoon/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


====== WebDriver manager ======


...

Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/camilleyoon/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


====== WebDriver manager ======


https://book.naver.com/category/index.naver?cate_code=100020&tab=top100&list_type=list&sort_type=publishday&page=4


Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/camilleyoon/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome


https://book.naver.com/category/index.naver?cate_code=100020&tab=top100&list_type=list&sort_type=publishday&page=5


Driver [/Users/camilleyoon/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [177]:
df

,순위,책이름,저자,줄거리
0,1,인간 실격,다자이 오사무,인간의 나약함을 탁월하게 묘사하는 다자이 오사무의 작품을 새롭게 읽는다. 순수하고 ...
1,2,데미안,헤르만 헤세,데미안을 통해 참다운 어른이 되어 가는 소년 싱클레어의 이야기.한 폭의 수채화같이 ...
2,3,1984,조지 오웰,"『동물농장』과 함께 조지 오웰을 대표하는 작품으로, 전제주의라는 거대한 지배 시스템..."
3,4,연금술사,파울로 코엘료,전세계 8천 5백만 독자가 읽은 전설적인 베스트셀러마음의 목소리에 귀기울이는 것이 ...
4,5,호밀밭의 파수꾼,제롬 데이비드 샐린저,1951년 처음 발표된 이래 가장 사랑받는(특히 젊은이들로부터) 고전의 자리를 놓치...
5,6,모모,미하엘 엔데,원작이 지닌 매력을 모두 되살린 한국어 판 <모모> 기적과 신비와 온기로 가득 찬 ...
6,7,자기 앞의 생,로맹 가리,"유일하게 공쿠르 상을 두 번 받은 작가 로맹 가리\r하밀 할아버지, 사람은 사랑 없..."
7,8,그리고 아무도 없었다,아가사 크리스티,추리소설의 여왕 애거서 크리스티 장편소설. 크리스티의 전작품 중에서 스릴과 서스펜스...
8,9,달과 6펜스,윌리엄 서머셋 모옴,프랑스의 후기 인상파 화가 폴 고갱을 모델로 한 중년의 사내(스트릭랙드)가 달빛 세...
9,10,오만과 편견,제인 오스틴,"원문에 충실한 정확한 번역으로 만나는 『오만과 편견』!전혀 새로운 번역, 원문에 충..."


In [168]:
df=pd.DataFrame(data=df)

In [170]:
df.columns=["순위", "책이름", "저자", "줄거리"]

In [172]:
df1=df

In [181]:
df1.to_csv('./data/zz.csv')

In [184]:
df

[[41,
  '백년 동안의 고독',
  '가브리엘 가르시아 마르케스',
  "라틴아메리카의 특수한 사회구조를 환상적으로 직조해낸 가브리엘 마르케스의 대표작!노벨문학상 수상작가 가브리엘 마르케스의 대표 장편소설 『백년 동안의 고독』. 1982년 노벨문학상 수상작으로, 중남미 문학의 특징인 '마술적 리얼리즘'의 원조격인 소설로 평가받는다. 마을에서 도시로 팽창하다가 신기루처럼 사라져간 마콘도를 무대로 한 집안의 백년에 걸친 역사와, 기이한 자녀를 에워싸고 일어나는 이야기를 엮은 작품이다. 라틴 아메리카의 복합적인 인종, 문화, 역사적 전통과 현실을 배경으로 하여, 라틴아메리카의 특수한 사회구조를 생생하게 그려냈다."],
 [42,
  '허삼관 매혈기',
  '위화',
  '설령 목숨을 파는 거라 해도 난 피를 팔아야 합니다!가족을 위해 피를 파는 한 남자의 고단한 삶을 특유의 풍자와 해학으로 그려낸 중국 작가, 위화 장편소설. 한평생 피를 팔아 가족을 위기에서 구해낸 속 깊은 아버지 허삼관을 주인공으로 한 소설로, 삶의 고단함과 슬픔을 능청스럽게 껴안는 익살과 해학 그리고 그 뒤에 자리한 인간에 대한 속 깊은 애정을 만날 수 있다. 성안의 생사공장에서 누에고치 대주는 일을 하는 노동자 허삼관. 그의 삼촌들이 사는 마을에서는 피를 안 팔아본 남자는 여자를 얻을 수 없다. 결혼의 조건으로 가장 중요한 것은 건강인데 피를 팔 수 있다는 것은 건강하다는 증거이기 때문이다. 그래서 허삼관은 그 마을 사람인 근룡이와 방씨를 따라 피를 팔러 성안의 병원으로 간다. 피를 팔러 가는 날은 아..'],
 [43,
  '인간의 굴레에서 1',
  '윌리엄 서머셋 모옴',
  '서머셋 몸의 <인간의 굴레에서>는 교양소설(Bildungsroman) 계열에 든다. 교양소설이란 젊은이가 인생과 사회에 눈떠가는 과정을 그린, 성장소설과도 비슷한 개념이다. 이 책 역시 고뇌를 짊어진 한 젊은이가 마음의 상처를 극복하면서 세상에 눈떠가는 과정을 섬세하게 그렸다.'],
 [44,
  '괭이부리말

In [185]:
len(df)

300

In [204]:
df2=pd.DataFrame(data=df,columns=["순위", "책이름", "저자", "줄거리"])

In [205]:
df2

,순위,책이름,저자,줄거리
0,1,생각의 지도,리처드 니스벳,"공자의 후손들과 아리스토텔레스의 후손들을 해부하는 비교문화 연구서. 교육, 과학, ..."
1,2,생각의 탄생,"미셸 루트번스타인, 로버트 루트번스타인",천재들이 활용한 창조적 사고의 13가지 도구들『생각의 탄생』은 분야를 넘나들며 창조...
2,3,예루살렘의 아이히만,한나 아렌트,나치의 전범 아돌프 아이히만\r\r1906년 독일 졸링겐에서 태어난 아이히만은 19...
3,4,욕망의 진화,데이비드 버스,남녀의 엇갈린 욕망에 숨겨진 진실을 파헤치다<욕망의 진화>는 인간 남녀의 서로 다른...
4,5,타인의 고통,수전 손택,오늘날 타인의 고통을 염려한다는 것은 어떤 의미일까? 지난 2003년 10월 12...
...,...,...,...,...
1041,1056,문화장벽을 넘어 더불어 살기,"김삼화, 김창대",최근 중국의 개혁개방정책에 따라 많은 중국학생들이 한국으로 유입되고 있다. 한국과 ...
1042,1057,중부 동해안 방언의 성조 비교,김차균,중부 동해안 방언의 성조를 비교 연구한 책. 방점법 성조 이론으로 강원 및 경북 동...
1043,1058,수업방법론과 교수법,루돌프 슈타이너,슈타이너의 교육학 시리즈 제4권. <교육 예술 Ⅰ>에서는 대안교육을 위한 수업방법론...
1044,1059,세계의 모든 얼굴,이정우,철학자 이정우의 현대 회화론<세계의 모든 얼굴>은 존재론과 연결시켜 현대 회화를 사...


In [206]:
df2.to_csv('./data/인문.csv')

In [192]:
df

[[361,
  '사서함 110호의 우편물',
  '이도우',
  "이 책은 한번쯤 실패를 맛본 30대 주인공들이 그래도 '다시 한번 사랑해 보기로'마음을 맞춰가는 가슴 따듯한 사랑이야기이다. 사소한 규칙이라도 당연한 듯 지키며 살아온 한 내성적인 여자가, 처음으로 먼저 '내게 문을 열어주세요'하고 노트해보는 사랑 이야기. 한번 두드려봤지만 쉽사리 열리지 않자 그녀는 얼른 없던 일로 하고 철수하려고 한다. 안 그래도 세상이 만만찮은데 어려운 사랑까지 보태고 싶지 않기 때문에... . 그런 그녀의 마음을 따라 잔잔하게 진행되는 이 글은 사랑하는 일에 능숙하지도 그렇다고 소홀하지도 않은 또 다른 많은 '그녀'들에게 진정한 용기와 사랑의 의미를 전해주고 있다.참신한 감성과 만만치 않은 문장력을 갖춘 이 작품은 수많은 독자들의 가슴을 두드리며 스테디셀러의 자리를 지켜왔으며 2007.."],
 [362,
  '앨저넌에게 꽃을',
  '대니얼 키스',
  'KBS TV <안녕하세요 하느님!>의 원작 소설 『앨저넌에게 꽃을』. IQ 70의 바보에서 IQ 180의 천재가 되어버린 빵가게 점원 찰리의 행복하고도 슬픈 날들을 그려내고 있다. 이 소설은 찰리의 보고서이자 일기 형식으로 구성되어 있다. 급격한 지능변화를 겪는 찰리는 천재가 되기까지 스스로 자신의 정신상태를 담담히 서술해 나가는데, 이러한 형식은 설득력 있게 다가오는 긴장감과 마력, 감동을 한층 더해준다. 성취지향적이고 물질 만능주의적인 사회 속에 인간의 존엄성과 가치가 매몰되는 과정을 그려내며, 인생에 있어 많은 지식의 섭렵만이 과연 최고의 가치인지를 다시 한번 생각하게 해준다.'],
 [363,
  '율리시스',
  '제임스 조이스',
  '모더니즘 문학의 대표작 『율리시스』완결판제임스 조이스의 대표작 『율리시스』완결판. 1968년과 1988년에 각각 나온 초역본과 재번역본에 이어, 제임스 조이스 문학 연구자인 김종건 고려대 교수의 새 번역본이다. 소설은 아일랜드의 수도 더블린을 배경으로 리오폴드 블룸, 그의 아

In [193]:
len(df)

600

In [194]:
df2=pd.DataFrame(data=df,columns=["순위", "책이름", "저자", "줄거리"])

In [195]:
df2.tail()

,순위,책이름,저자,줄거리
595,956,말콤 엑스 상,알렉스 헤일리,"구두닦이, 양아치, 강도, 웨이터, 마약밀매꾼, 뚜장이 등 밑바닥 생활에서 몸을 일..."
596,957,꿈을 빌려드립니다,가브리엘 가르시아 마르케스,'마술적 사실주의'의 대표적인 작가인 가브리엘 가르시아 마르케스의 작품집. 작가의 ...
597,958,하얀성,오르한 파묵,원전 번역으로 만나는 거장 오르한 파묵의 기념비적 역작! 21세기가 주목하는 작가 ...
598,959,구운몽 사씨남정기,김만중,서포 김만중의 현존 소설작품 <구운몽>과 <사씨남정 기>를 한데 묶었다.
599,960,허균 최후의 19일 세트,김탁환,허균의 혁명적 활동을 두터운 학구와 통렬한 실감으로재해석한 역사소설. 마흔 다섯의 ...


In [196]:
df2.to_csv('./data/순위960.csv')